# Reproducing Tables 7, 8, and 9 from PINT paper

This script allows one to create the tables in the PINT paper which measure the runtime of basic PINT functionality and compare with TEMPO/Tempo2 runtimes.

* Requirements
    * TEMPO
    * Tempo2
    * Needs to be run from the same location as data files

In [ ]:
from pint import toa
from pint import models
from pint import fitter
import subprocess
import timeit
from os import path
from astropy.table import Table
from astropy.io import ascii

### Setup

* Define number of iterations to average for each run (5 iterations in paper)
* Establish # of TOAs and .par/.tim files

In [ ]:
MAXIT = 5  # number of iterations to time and average
# number of TOAs run for each test
ntoas_simple = [100, 1000, 10000, 100000]
ntoas_complex = [5012, 10024, 25060]
par_simple = "NGC6440E.par"
par_complex = "J1910+1256_NANOGrav_12yv4.gls.par"
timfiles_simple = [
    "NGC6440E_fake100.tim",
    "NGC6440E_fake1000.tim",
    "NGC6440E_fake10000.tim",
    "NGC6440E_fake100000.tim",
]  # timfiles for simple model and individual functions
timfiles_complex = [
    "J1910+1256_NANOGrav_12yv4.tim",
    "J1910+1256_NANOGrav_12yv4_10k.tim",
    "J1910+1256_NANOGrav_12yv4_25k.tim",
]  # timfiles for complex model

### Create function to run and time PINT

In [ ]:
def pintrun(parfile, timfile, ptime_arr, pickle, fitter):
    """ Runs and times pintempo 5 times and averages times, appending to a list. """
    total = 0
    if fitter == "gls" and pickle:
        for i in range(MAXIT):
            start = timeit.default_timer()
            subprocess.check_call(
                ["pintempo", "--usepickle", "--gls", parfile, timfile],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
            )
            end = timeit.default_timer()
            total = total + (end - start)
    if fitter == "gls" and not pickle:
        for i in range(MAXIT):
            start = timeit.default_timer()
            subprocess.check_call(
                ["pintempo", "--gls", parfile, timfile],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
            )
            end = timeit.default_timer()
            total = total + (end - start)
    if fitter != "gls" and pickle:
        for i in range(MAXIT):
            start = timeit.default_timer()
            subprocess.check_call(
                ["pintempo", "--usepickle", parfile, timfile],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
            )
            end = timeit.default_timer()
            total = total + (end - start)
    if fitter != "gls" and not pickle:
        for i in range(MAXIT):
            start = timeit.default_timer()
            subprocess.check_call(
                ["pintempo", parfile, timfile],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
            )
            end = timeit.default_timer()
            total = total + (end - start)
    ptime_arr.append(total / MAXIT)  # averages time

### Create function to run and time TEMPO

In [ ]:
def temporun(parfile, timfile, ttime_arr, fitter):
    """ Runs and times TEMPO 5 times and averages times, appending to a list. """
    total = 0
    if fitter == "gls":
        for i in range(MAXIT):
            start = timeit.default_timer()
            subprocess.check_call(
                ["tempo", "-G", "-f", parfile, timfile],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
            )
            end = timeit.default_timer()
            total = total + (end - start)
    else:
        for i in range(MAXIT):
            start = timeit.default_timer()
            subprocess.check_call(
                ["tempo", "-f", parfile, timfile],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
            )
            end = timeit.default_timer()
            total = total + (end - start)
    ttime_arr.append(total / MAXIT)  # average time

### Create function to run and time Tempo2

In [ ]:
def tempo2run(parfile, timfile, t2time_arr):
    """ Runs and times Tempo2 5 times and averages times, appending to a list. """
    total = 0
    for i in range(MAXIT):
        start = timeit.default_timer()
        subprocess.check_call(
            ["tempo2", "-nobs", "100003", "-f", parfile, timfile],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
        )
        end = timeit.default_timer()
        total = total + (end - start)
    t2time_arr.append(total / MAXIT)  # average time

### Create fake TOAs for simple model use

* If this block has already been run and files are still in current directory, can skip for future runs

In [ ]:
# Generate simple, fake TOAs for the timing runs
print("Making fake TOAs...")
for num in ntoas_simple:
    call = ["zima", "--startMJD", "53478", "--duration", "700", "--freq", "1400", "2000", "--ntoa", str(num), par_simple, "NGC6440E_fake" + str(num) + ".tim"]
    if path.exists("NGC6440E_fake" + str(num) + ".tim"):
        pass
    else:
        subprocess.check_call(call)
print("Done.")

### Run and time simple model case

* Creates Table 7 in PINT paper

In [ ]:
ptimes_nopickle = []
ptimes_pickle = []
ttimes = []
t2times = []

for tim in timfiles_simple:
    print("With " + tim + "...")
    print("Running PINT fitting w/o pickling...")
    # run PINT w/o pickling and average time over 5 runs
    pintrun(par_simple, tim, ptimes_nopickle, pickle=False, fitter="wls")

    print("Running PINT w/ pickling...")
    # run PINT with pickling and average time over 5 runs
    subprocess.check_call(
        ["pintempo", "--usepickle", par_simple, tim],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )  # create pickle file
    pintrun(par_simple, tim, ptimes_pickle, pickle=True, fitter="wls")
    print("Running TEMPO...")
    temporun(par_simple, tim, ttimes, fitter="wls")
    print("Running Tempo2...")
    tempo2run(par_simple, tim, t2times)

# create table 7 in PINT paper
simple_comparison = Table(
    (ntoas_simple, ttimes, t2times, ptimes_nopickle, ptimes_pickle),
    names=(
        "Number of TOAs",
        "TEMPO (sec)",
        "Tempo2 (sec)",
        "PINT - No Pickle (sec)",
        "PINT - Pickle (sec)",
    ),
)
print("Done")

In [ ]:
# Write out results
ascii.write(
    simple_comparison,
    "simple_tables.tex",
    Writer=ascii.AASTex,
    formats={"TEMPO (sec)": '%.3f', "Tempo2 (sec)": '%.3f', "PINT - No Pickle (sec)": '%.3f',
        "PINT - Pickle (sec)": '%.3f'},
    overwrite=True,
)



### Run and time individual functions in PINT

* Creates Table 8 in PINT paper

In [ ]:
# time the individual PINT functions
importtimes = []
getTOAs_nopickle = []
getTOAs_pickle = []
fittimes = []

# time import statements
total = 0
for i in range(MAXIT):
    start = timeit.default_timer()
    subprocess.check_call(
        ["python3", "import_statements.py"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )
    end = timeit.default_timer()
    total = total + (end - start)
for i in range(len(ntoas_simple)):
    importtimes.append(total / MAXIT)

# setup
m = models.get_model(par_simple)
for tim in timfiles_simple:
    # no pickle time of get_TOAs
    print("timing get_TOAs w/o pickling...")
    total = 0
    for i in range(MAXIT):
        start = timeit.default_timer()
        toa.get_TOAs(
            tim, model=m, usepickle=False
        )
        end = timeit.default_timer()
        total = total + (end - start)
    getTOAs_nopickle.append(total / MAXIT)

    t = toa.get_TOAs(
        tim, model=m, usepickle=True
    )  # to use in timing fitter

    f = fitter.WLSFitter(t, m)

    print("timing fitter...")
    total = 0
    for i in range(MAXIT):
        start = timeit.default_timer()
        f.fit_toas()
        end = timeit.default_timer()
        total = total + (end - start)
    fittimes.append(total / MAXIT)

    # pickle time of get_TOAs
    print("timing get_TOAs w/ pickling...")
    total = 0
    for i in range(MAXIT):
        start = timeit.default_timer()
        toa.get_TOAs(
            tim, model=m, usepickle=True
        )
        end = timeit.default_timer()
        total = total + (end - start)
    getTOAs_pickle.append(total / MAXIT)

# create table 8 in PINT paper
function_comparison = Table(
    (ntoas_simple, importtimes, getTOAs_nopickle, getTOAs_pickle, fittimes),
    names=(
        "Number of TOAs",
        "Import Statements (sec)",
        "Load TOAs - No Pickle (sec)",
        "Load TOAs - Pickle (sec)",
        "WLS Fitting - No Pickle (sec)",
    ),
)
print("Done.")

In [ ]:
# Write out table
ascii.write(
    function_comparison,
    "function_tables.tex",
    Writer=ascii.AASTex,
    formats={"Import Statements (sec)": '%.3f', "Load TOAs - No Pickle (sec)": '%.3f', "Load TOAs - Pickle (sec)": '%.3f',
        "WLS Fitting - No Pickle (sec)": '%.3f'},    overwrite=True,
)

### Create .tim files to run complex model with different number of TOAs

* Can skip block if files from previous runs still in directory

* The .par file for J1910+1256 in this directory has the following additional parameters from the original dataset file (to ensure GLS fit with Tempo2):
    * TNRedAmp -14.227505410948254
    * TNRedGam 4.91353
    * TNRedC 45

In [ ]:
# copy TOAs to create 2x the number of TOAs
with open("J1910+1256_NANOGrav_12yv4.tim") as original:
    with open("J1910+1256_NANOGrav_12yv4_10k.tim", "w") as new:
        # copy entire file
        for line in original:
            new.write(line)
            if "MODE" in line or "FORMAT" in line:
                pass
            else:
                new.write(line)  # copy TOAs to create 2x number of TOAs
# copy TOAs to create 5x the number of TOAs
with open("J1910+1256_NANOGrav_12yv4.tim") as original:
    with open("J1910+1256_NANOGrav_12yv4_25k.tim", "w") as new:
        # copy entire file
        for line in original:
            new.write(line)
            if "MODE" in line or "FORMAT" in line:
                pass
            else:
                new.write(line)  # copy TOAs to create 5x number of TOAs
                new.write(line)
                new.write(line)
                new.write(line)

### Run and time complex model case

* Creates Table 9 in PINT paper

In [ ]:
ptimes_nopickle2 = []
ptimes_pickle2 = []
ttimes2 = []
t2times2 = []

for tim in timfiles_complex:
    print("With " + tim + "...")
    # run PINT w/o pickling and average time over 5 runs
    print("Running PINT w/o pickling...")
    pintrun(par_complex, tim, ptimes_nopickle2, pickle=False, fitter="gls")

    # run PINT with pickling and average time over 5 runs
    print("Running PINT w/ pickling...")
    subprocess.check_call(
        ["pintempo", "--usepickle", par_complex, tim],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )  # create pickle file
    pintrun(par_complex, tim, ptimes_pickle2, pickle=True, fitter="gls")

    print("running TEMPO...")
    temporun(par_complex, tim, ttimes2, fitter="gls")

    print("running Tempo2...")
    tempo2run(par_complex, tim, t2times2)

# create table 9 in PINT paper
complex_comparison = Table(
    (ntoas_complex, ttimes2, t2times2, ptimes_nopickle2, ptimes_pickle2),
    names=(
        "Number of TOAs",
        "TEMPO (sec)",
        "Tempo2 (sec)",
        "PINT - No Pickle (sec)",
        "PINT - Pickle (sec)",
    ),
)
print("Done.")

In [ ]:
# Write out results 
ascii.write(
    complex_comparison,
    "complex_tables.tex",
    Writer=ascii.AASTex,
    formats={"TEMPO (sec)": '%.3f', "Tempo2 (sec)": '%.3f', "PINT - No Pickle (sec)": '%.3f',
        "PINT - Pickle (sec)": '%.3f'},
    overwrite=True,
)
